<a href="https://colab.research.google.com/github/yshin1209/Awesome-CGM/blob/master/Glucose_Prediction_Simple_Working_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train empty GPT-2 for glucose level prediction.

In [1]:
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel


In [7]:
# Preprocess the training data
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
train_data = [" 87 97 105 117 123 132 154 176 192", " 100 98 106 118 124 131 156 179 193", " 102 99 108 113 129 132 156 178 199"]
input_ids = [tokenizer.encode(text) for text in train_data]
# print(f'input_ids {input_ids}')
inputs = torch.tensor(input_ids)
# print(inputs)

# Set up the GPT-2 model configuration
model_config = GPT2Config(
    vocab_size=len(tokenizer),
    n_embd=300,
    n_layer=36,
    n_head=12,
    attn_pdrop=0.5,
    resid_pdrop=0.5,
    initializer_range=0.02,
    repetition_penalty=1,  # No penalty
    temperature = 0
)

# Set up the GPT-2 model with language modeling head
model = GPT2LMHeadModel(model_config)

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
for epoch in range(3):
    model.train()
    total_loss = 0
    for input_seq in inputs:
        print (f'total_loss b: {total_loss}')
        optimizer.zero_grad()
        print (f'total_loss a: {total_loss}')
        input_seq = input_seq.unsqueeze(0)  # Reshape to 2D tensor
        outputs = model(input_seq[:, :-1], labels=input_seq[:, 1:])
        #print (f'outputs {outputs}')
        loss = outputs.loss
        # print (f'loss: {loss}')
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1} loss: {total_loss / len(inputs)}")



total_loss b: 0
total_loss a: 0
total_loss b: 10.843235969543457
total_loss a: 10.843235969543457
total_loss b: 21.87674903869629
total_loss a: 21.87674903869629
Epoch 1 loss: 10.762398719787598
total_loss b: 0
total_loss a: 0
total_loss b: 9.475279808044434
total_loss a: 9.475279808044434
total_loss b: 19.312416076660156
total_loss a: 19.312416076660156
Epoch 2 loss: 9.478894233703613
total_loss b: 0
total_loss a: 0
total_loss b: 8.804800987243652
total_loss a: 8.804800987243652
total_loss b: 17.831022262573242
total_loss a: 17.831022262573242
Epoch 3 loss: 8.976401646931967


In [6]:
# Make a prediction
text = " 88 96"
input_ids = tokenizer.encode(text, return_tensors='pt')
outputs = model.generate(input_ids, max_length=9)
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(predicted_text)
# 87 97 105 117 123 132 154 176 192
# 88 96 118 124 132 156 178 192 192 (temp 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 88 96 132 156 132 156 156 156 156
